In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import random
import quaternion as qt

import habitat_sim
from habitat_sim import AgentConfiguration, CameraSensorSpec

from tbp.monty.simulators.habitat import HabitatSim
from tbp.monty.frameworks.environments.habitat import PanTiltZoomCamera
from tbp.monty.frameworks.environment_utils.transforms import DepthTo3DLocations

from graph_utils import *

import warnings
warnings.filterwarnings('ignore')

import torch
import numpy as np
from collections import defaultdict
from itertools import count
import time
from IPython.display import clear_output

from nupic.research.frameworks.columns import ApicalTiebreakSequenceMemory

In [2]:
def circle_train(timestep, on_size=5, padding=(2, 2), length=50):
    sdr = np.zeros(length)
    
    start = (on_size * timestep) % length - padding[0]
    end = (on_size * timestep) % length + on_size + padding[1]
    
    if start < 0:
        sdr[start:] = 1
        start = 0
    if end >= length:
        sdr[:end-length] = 1
        end = length
    
    sdr[start:end] = 1
    
    return sdr.astype(np.uint8)

def circle_test(timestep, on_size=5, offset=2, length=50):
    sdr = np.zeros(length)
    
    start = (on_size * timestep) % length + offset
    end = (on_size * timestep) % length + on_size + offset
    
    if start < 0:
        sdr[start:] = 1
        start = 0
    if end >= length:
        sdr[:end-length] = 1
        end = length
    
    sdr[start:end] = 1
    
    return sdr.astype(np.uint8)


def formatRow(x):
    s = ""
    for c in range(len(x)):
        if c > 0 and c % 10 == 0:
            s += " "
        s += str(x[c])
    s += " "
    return s

def accuracy(active, predicted):
    accuracy = 0
    
    if np.count_nonzero(predicted) > 0:
        accuracy = float(np.dot(active, predicted)) / float(np.count_nonzero(predicted))
    
    return accuracy

In [13]:
tm = ApicalTiebreakSequenceMemory(
    columnCount=50,
    cellsPerColumn=2,
    initialPermanence=0.5,
    minThreshold=5,
    permanenceIncrement=0.1,
    permanenceDecrement=0.0,
    activationThreshold=5,
)

"""
TRAIN TM
"""

for i in range(10):
    # Send each letter in the sequence in order
    for j in range(9):
        active_columns = list(set([i for i, j in zip(count(), circle_train(j, padding=(2, 2))) if j == 1]))

        # The compute method performs one step of learning and/or inference. Note:
        # here we just perform learning but you can perform prediction/inference and
        # learning in the same step if you want (online learning).
        tm.compute(active_columns, learn=True)

        print("seeing pattern", str(list(range(10))[j]))
        print("active cells " + str(tm.getActiveCells()))
        print("predictive cells " + str(tm.getPredictedCells()))
        print("winner cells " + str(tm.getWinnerCells()))
        print("# of active segments " + str(tm.basalConnections.nSegments()))
        #print()
        
        active_column_indices = [int(i / tm.cellsPerColumn) for i in tm.getActiveCells()]
        predicted_column_indices = [int(i / tm.cellsPerColumn) for i in tm.getPredictedCells()]

        # Reconstructing the active and inactive columns with 1 as active and 0 as
        # inactive representation.

        active_column_state = [
            "1" if i in active_column_indices else "0" for i in range(tm.numberOfColumns())
        ]
        active_column_str = "".join(active_column_state)
        all_active_columns = np.array([int(each) for each in active_column_state])


        predictive_column_state = [
            "1" if i in predicted_column_indices else "0" for i in range(tm.numberOfColumns())
        ]
        predictive_column_str = "".join(predictive_column_state)
        all_predictive_columns = np.array([int(each) for each in predictive_column_state])
        
        print("Accuracy: " + str(accuracy(all_active_columns, all_predictive_columns)) + "\n")


    # The reset command tells the TM that a sequence just ended and essentially
    # zeros out all the states. It is not strictly necessary but it's a bit
    # messier without resets, and the TM learns quicker with resets.
    tm.reset()
    print("Reset!")

seeing pattern 0
active cells [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 96 97 98 99]
predictive cells []
winner cells [ 0  2  5  6  8 11 12 97 99]
# of active segments 0
Accuracy: 0

seeing pattern 1
active cells [ 6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
predictive cells []
winner cells [ 6  8 10 13 14 17 19 20 23]
# of active segments 9
Accuracy: 0

seeing pattern 2
active cells [16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33]
predictive cells []
winner cells [16 18 21 22 25 27 29 30 33]
# of active segments 18
Accuracy: 0

seeing pattern 3
active cells [26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43]
predictive cells []
winner cells [26 28 31 32 34 36 38 40 42]
# of active segments 27
Accuracy: 0

seeing pattern 4
active cells [36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53]
predictive cells []
winner cells [37 39 41 43 45 47 49 51 53]
# of active segments 36
Accuracy: 0

seeing pattern 5
active cells [46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 6

In [16]:
"""
TEST TM (with same sequences)
"""

for j in range(9):
    print("\n\n--------", str(list(range(9))[j]), "-----------")
    print("Raw input vector : " + formatRow(circle_train(j)))
    active_columns = list(set([i for i, j in zip(count(), circle_train(j)) if j == 1]))

    # Send each vector to the TM, with learning turned off
    tm.compute(active_columns, learn=False)

    # The following print statements prints out the active cells, predictive
    # cells, active segments and winner cells.
    #
    # What you should notice is that the columns where active state is 1
    # represent the SDR for the current input pattern and the columns where
    # predicted state is 1 represent the SDR for the next expected pattern
    print("\nAll the active and predicted cells:")

    print("active cells " + str(tm.getActiveCells()))
    print("predictive cells " + str(tm.getPredictedCells()))
    print("winner cells " + str(tm.getWinnerCells()))
    print("# of active segments " + str(tm.basalConnections.nSegments()))

    active_column_indices = [int(i / tm.cellsPerColumn) for i in tm.getActiveCells()]
    predicted_column_indices = [int(i / tm.cellsPerColumn) for i in tm.getPredictedCells()]

    # Reconstructing the active and inactive columns with 1 as active and 0 as
    # inactive representation.

    active_column_state = [
        "1" if i in active_column_indices else "0" for i in range(tm.numberOfColumns())
    ]
    active_column_str = "".join(active_column_state)
    all_active_columns = np.array([int(each) for each in active_column_state])
    
    
    predictive_column_state = [
        "1" if i in predicted_column_indices else "0" for i in range(tm.numberOfColumns())
    ]
    predictive_column_str = "".join(predictive_column_state)
    all_predictive_columns = np.array([int(each) for each in predictive_column_state])

    # For convenience the cells are grouped
    # 10 at a time. When there are multiple cells per column the printout
    # is arranged so the cells in a column are stacked together
    print("Active columns:    " + formatRow(active_column_str))
    print("Predicted columns: " + formatRow(predictive_column_str))
    print("Accuracy: " + str(accuracy(all_active_columns, all_predictive_columns)))



-------- 0 -----------
Raw input vector : 1111111000 0000000000 0000000000 0000000000 0000000011 

All the active and predicted cells:
active cells [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 96 97 98 99]
predictive cells []
winner cells [ 1  3  5  7  9 11 12 97 98]
# of active segments 72
Active columns:    1111111000 0000000000 0000000000 0000000000 0000000011 
Predicted columns: 0000000000 0000000000 0000000000 0000000000 0000000000 
Accuracy: 0


-------- 1 -----------
Raw input vector : 0001111111 1100000000 0000000000 0000000000 0000000000 

All the active and predicted cells:
active cells [ 6  8 10 13 14 17 19 20 23]
predictive cells [ 6  8 10 13 14 17 19 20 23]
winner cells [ 6  8 10 13 14 17 19 20 23]
# of active segments 72
Active columns:    0001111111 1100000000 0000000000 0000000000 0000000000 
Predicted columns: 0001111111 1100000000 0000000000 0000000000 0000000000 
Accuracy: 1.0


-------- 2 -----------
Raw input vector : 0000000011 1111111000 0000000000 0000000000 000

In [17]:
"""
TEST TM (with different sequences)
"""

def test_tm(seq_start):
    tm.reset() 
    
    for j in range(seq_start, seq_start + 2):
        if j == seq_start:
            print("\n\n--------", str(list(range(10))[j]) + "'", "-----------")
            #print("Raw input vector : " + formatRow(circle_test(j, offset=3)))
            active_columns = list(set([i for i, j in zip(count(), circle_test(j, offset=2)) if j == 1]))
        else:
            print("\n\n--------", str(list(range(10))[j]), "-----------")
            #print("Raw input vector : " + formatRow(circle_train(j)))
            active_columns = list(set([i for i, j in zip(count(), circle_train(j, padding=(2,2))) if j == 1]))

        # Send each vector to the TM, with learning turned off
        tm.compute(active_columns, learn=False)

        # The following print statements prints out the active cells, predictive
        # cells, active segments and winner cells.
        #
        # What you should notice is that the columns where active state is 1
        # represent the SDR for the current input pattern and the columns where
        # predicted state is 1 represent the SDR for the next expected pattern
        #print("\nAll the active and predicted cells:")

        #print("active cells " + str(tm.getActiveCells()))
        #print("predictive cells " + str(tm.getPredictedCells()))
        #print("winner cells " + str(tm.getWinnerCells()))
        #print("# of active segments " + str(tm.basalConnections.nSegments()))

        active_column_indices = [int(i / tm.cellsPerColumn) for i in tm.getActiveCells()]
        predicted_column_indices = [int(i / tm.cellsPerColumn) for i in tm.getPredictedCells()]

        # Reconstructing the active and inactive columns with 1 as active and 0 as
        # inactive representation.

        active_column_state = [
            "1" if i in active_column_indices else "0" for i in range(tm.numberOfColumns())
        ]
        active_column_str = "".join(active_column_state)
        all_active_columns = np.array([int(each) for each in active_column_state])


        predictive_column_state = [
            "1" if i in predicted_column_indices else "0" for i in range(tm.numberOfColumns())
        ]
        predictive_column_str = "".join(predictive_column_state)
        all_predictive_columns = np.array([int(each) for each in predictive_column_state])

        # For convenience the cells are grouped
        # 10 at a time. When there are multiple cells per column the printout
        # is arranged so the cells in a column are stacked together
        print("Active columns:    " + formatRow(active_column_str))
        print("Predicted columns: " + formatRow(predictive_column_str))
        print("Accuracy: " + str(accuracy(all_active_columns, all_predictive_columns)))

In [18]:
test_tm(seq_start=0)
print()

test_tm(seq_start=1)
print()

test_tm(seq_start=2)
print()

test_tm(seq_start=3)
print()

test_tm(seq_start=4)
print()

test_tm(seq_start=5)
print()

test_tm(seq_start=6)
print()

test_tm(seq_start=7)
print()



-------- 0' -----------
Active columns:    0011111000 0000000000 0000000000 0000000000 0000000000 
Predicted columns: 0000000000 0000000000 0000000000 0000000000 0000000000 
Accuracy: 0


-------- 1 -----------
Active columns:    0001111111 1100000000 0000000000 0000000000 0000000000 
Predicted columns: 0001111111 1100000000 0000000000 0000000000 0000000000 
Accuracy: 1.0



-------- 1' -----------
Active columns:    0000000111 1100000000 0000000000 0000000000 0000000000 
Predicted columns: 0000000000 0000000000 0000000000 0000000000 0000000000 
Accuracy: 0


-------- 2 -----------
Active columns:    0000000011 1111111000 0000000000 0000000000 0000000000 
Predicted columns: 0000000011 1111111000 0000000000 0000000000 0000000000 
Accuracy: 1.0



-------- 2' -----------
Active columns:    0000000000 0011111000 0000000000 0000000000 0000000000 
Predicted columns: 0000000000 0000000000 0000000000 0000000000 0000000000 
Accuracy: 0


-------- 3 -----------
Active columns:    0000000000 0